## Fine-tune AraBERTv2 on Custom Dataset

This is a POC for Fine-tuning AraBERTv2 or other available pre-trained models: https://huggingface.co/aubmindlab on a Custom Dataset.

In [ ]:
from src.finetuningQA.data_split import train_dev_test_split
import json
import datasets
import os

In [ ]:
# arabert2 model local path. 
# You can download another pretrained models from here: https://huggingface.co/aubmindlab
model_path= "bert-base-arabertv2/"
model_name = "aubmindlab/bert-base-arabertv2"

!wget https://huggingface.co/aubmindlab/bert-base-arabertv2/resolve/main/pytorch_model.bin -O bert-base-arabertv2/pytorch_model.bin


## Splitt the dataset into train/dev/test (if needed)

In [3]:
#train_dev_test_split("dataset/question-answering-dataset/iskan_train.json")

## Pre-process the data
Here we will prepare the dataset in the format that the model expect.

The Dataset structure must be as follow:

'''

    {"data": 

        [ 
            {"paragraphs": 

                [
                    {"qas": 
            
                    [{"question": "ما هو برنامج مزايا", "id": 445985, 

                        "answers": ["text": "هو أحد المبادرات الجديدة التي تطرحها وزارة الإسكان والتخطيط العمراني بالتعاون مع القطاع الخاص لتوفير السكن الاجتماعي للمواطنين المدرجة أسمائهم على قوائم الانتظار"}, 
            
                    {"question": "ما هو اقصى حد لقيمة القسط الشهري المترتب على المواطن؟", "id": 445986, 
                
                        "answers": ["text": "25% من راتب المواطن كحد أقصى."]

                    "context": "نبذة عن برنامج "مزايا " برنامج "مزايا " هو أحد المبادرات الجديدة التي تطرحها وزارة الإسكان والتخطيط العمراني بالتعاون مع القطاع الخاص لتوفير السكن الاجتماعي للمواطنين المدرجة أسمائهم على قوائم الانتظار، وتقوم فكرة البرنامج على قيام المواطن الذي تنطبق عليه المعايير بشراء وحدة سكنية من خلال حصوله على تمويل من أحد البنوك المشاركة، على أن تقوم الحكومة بتوفير الدعم المالي للمواطن، والمتمثل في سداد الفارق بين قيمة القسط الفعلي لمبلغ التمويل المحدد من قبل البنك الممول، وقيمة القسط الشهري المستحق على المواطن والذي لا يتجاوز 25% من راتب المواطن كحد أقصى. ",
            "document_id": 862498"}
                ]
            }
        ]

    }

        
'''


In [ ]:
# Training Dataset
!python src/finetuningQA/preprocessing.py \
  --input_file="dataset/question-answering-dataset/iskan_new.json" \
  --output_file="dataset/question-answering-dataset/iskan_new_preprocessed.json" \
  --model_name=$model_name


# Eval Dataset
'''
!python src/finetuningQA/preprocessing.py \
  --input_file="dataset/question-answering-dataset/iskan_dev.json" \
  --output_file="dataset/question-answering-dataset/iskan_dev_preprocessed.json" \
  --model_name=$model_name
'''


In [16]:
# Load the preprocessed dataset and print one sample to see how the preprocessed data looks like
with open("dataset/question-answering-dataset/iskan_dev_preprocessed.json") as f:
 data = json.load(f)['data']

print(data[3])

{'paragraphs': [{'qas': [{'question': 'متى تم تأسيس لجن +ة ال+ إسكان و+ ال+ تمليك', 'id': 452838, 'answers': [{'answer_id': 559363, 'document_id': 976441, 'question_id': 452838, 'text': 'عام 1962', 'answer_start': 285, 'answer_end': 231, 'answer_category': None}], 'is_impossible': False}, {'question': 'ما هي ال+ مدين +ة ال+ أولى التي تم إنشائ +ها ل+ تلبي +ة حاج +ات ال+ مواطن +ين', 'id': 452839, 'answers': [{'answer_id': 559364, 'document_id': 976441, 'question_id': 452839, 'text': 'مدين +ة عيسى', 'answer_start': 436, 'answer_end': 350, 'answer_category': None}], 'is_impossible': False}], 'context': 'يعد ملف ال+ إسكان ب+ مملك +ة ال+ بحرين من أبرز ال+ ملف +ات ال+ معيشي +ة التي تحظى ب+ ال+ أولوي +ة لدى ال+ مواطن ال+ بحريني ، و+ هو ما وع +ت إلي +ه ال+ قياد +ة ال+ رشيد +ة منذ عقود طويل +ة ، و+ تحديد +ا منذ مطلع ستيني +ات ال+ قرن ال+ ماضي ، عندما تم تأسيس لجن +ة ال+ إسكان و+ ال+ تمليك عام 1962 ، و+ التي وضع +ت ال+ تشريع ال+ أول ال+ منظم ل+ سياس +ة تقديم ال+ خدم +ات ال+ إسكاني +ة ل+ ال+ مواطن

## Custom Dataset Loader

In [17]:
%%writefile src/finetuningQA/custom_dataset_loader.py
from __future__ import absolute_import, division, print_function

import json
import logging

import datasets


_CITATION = """\
@inproceedings{mozannar-etal-2019-neural,
    title = {Neural {A}rabic Question Answering},
    author = {Mozannar, Hussein  and Maamary, Elie  and El Hajal, Karl  and Hajj, Hazem},
    booktitle = {Proceedings of the Fourth Arabic Natural Language Processing Workshop},
    month = {aug},
    year = {2019},
    address = {Florence, Italy},
    publisher = {Association for Computational Linguistics},
    url = {https://www.aclweb.org/anthology/W19-4612},
    doi = {10.18653/v1/W19-4612},
    pages = {108--118},
    abstract = {This paper tackles the problem of open domain factual Arabic question answering (QA) using Wikipedia as our knowledge source. This constrains the answer of any question to be a span of text in Wikipedia. Open domain QA for Arabic entails three challenges: annotated QA datasets in Arabic, large scale efficient information retrieval and machine reading comprehension. To deal with the lack of Arabic QA datasets we present the Arabic Reading Comprehension Dataset (ARCD) composed of 1,395 questions posed by crowdworkers on Wikipedia articles, and a machine translation of the Stanford Question Answering Dataset (Arabic-SQuAD). Our system for open domain question answering in Arabic (SOQAL) is based on two components: (1) a document retriever using a hierarchical TF-IDF approach and (2) a neural reading comprehension model using the pre-trained bi-directional transformer BERT. Our experiments on ARCD indicate the effectiveness of our approach with our BERT-based reader achieving a 61.3 F1 score, and our open domain system SOQAL achieving a 27.6 F1 score.}
}
"""

_DESCRIPTION = """\
 Custom Dataset Loader 
"""

_URL = "../../dataset/question-answering-dataset/"
_URLs = {
    "train": _URL + "iskan_new_preprocessed.json",
    "dev": _URL + "iskan_new_preprocessed.json",
}



class CustomDatasetAraBERTConfig(datasets.BuilderConfig):
    """BuilderConfig for Custom Dataset."""

    def __init__(self, **kwargs):
        """BuilderConfig for CustomDataset.

        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(CustomDatasetAraBERTConfig, self).__init__(**kwargs)


class CustomDatasetAraBERT(datasets.GeneratorBasedBuilder):
    """CustomDataset"""

    BUILDER_CONFIGS = [
        CustomDatasetAraBERTConfig(
            name="plain_text",
            version=datasets.Version("1.0.0", ""),
            description="Plain text",
        )
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "title": datasets.Value("string"),
                    "context": datasets.Value("string"),
                    "question": datasets.Value("string"),
                    "answers": datasets.features.Sequence(
                        {"text": datasets.Value("string"), "answer_start": datasets.Value("int32")}
                    ),
                }
            ),
            # No default supervised_keys (as we have to pass both question
            # and context as input).
            supervised_keys=None,
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        urls_to_download = _URLs
        downloaded_files = dl_manager.download_and_extract(urls_to_download)

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": downloaded_files["train"]}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": downloaded_files["dev"]}),
        ]

    def _generate_examples(self, filepath):
        """This function returns the examples in the raw (text) form."""
        logging.info("generating examples from = %s", filepath)
        with open(filepath, encoding="utf-8") as f:
            data_ = json.load(f)
            for article in data_["data"]:
                title = article.get("title", "").strip()
                for paragraph in article["paragraphs"]:
                    context = paragraph["context"].strip()
                    for qa in paragraph["qas"]:
                        question = qa["question"].strip()

                        id_ = qa["id"]

                        answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                        answers = [answer["text"].strip() for answer in qa["answers"]]                     

                        # Features currently used are "context", "question", and "answers".
                        # Others are extracted here for the ease of future expansions.
                        yield id_, {
                            "title": title,
                            "context": context,
                            "question": question,
                            "id": id_,
                            "answers": {"answer_start": answer_starts, "text": answers},
                        }


Overwriting src/finetuningQA/custom_dataset_loader.py


In [18]:
iskan = datasets.load_dataset("src/finetuningQA/custom_dataset_loader.py")
print(iskan)

Reusing dataset custom_dataset_loader (/home/mohammed/.cache/huggingface/datasets/custom_dataset_loader/plain_text/1.0.0/cd8bb2354a58e1b2546ed6aade8861259cb6876a21e2f09793fede249feae5df)
100%|██████████| 2/2 [00:00<00:00,  9.71it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 28
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 28
    })
})


## Fine-tuning 

In [ ]:
!python src/finetuningQA/run_qa.py \
  --model_name_or_path $model_path \
  --dataset_name src/finetuningQA/custom_dataset_loader.py \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 2 \
  --gradient_accumulation_steps 1 \
  --learning_rate 3e-5 \
  --num_train_epochs 20 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ./run \
  --n_best_size 20 \
  --evaluation_strategy epoch \
  --save_steps 1000 \
  --seed 666 \
  --overwrite_output_dir \
  --warmup_steps 50

## Move the new pretrained model to the "arabertv2-QA-model/" to be able to run inference with it

In [25]:
os.replace("run/pytorch_model.bin","arabertv2-QA-model/pytorch_model.bin")
os.replace("run/tokenizer.json","arabertv2-QA-model/tokenizer.json")

## Run Inference

In [26]:
context_1 = (
  'يعد ملف الإسكان بمملكة البحرين من أبرز الملفات المعيشة'
  'التي تحظى بالأولوية لدى المواطن البحريني، وهو ما وعت إليه'
  'القيادة الرشيدة منذ عقود طويلة، وتحديداً منذ مطلع ستينيات القرن الماضي، عندما تم تأسيس لجنة'
  'الإسكان والتمليك عام 1962، والتي وضعت التشريع الأول المنظم لسياسة تقديم الخدمات'
  'الإسكانية للمواطنين، ليعقب ذلك بعام واحد إنشاء مشروع مدينة عيسى كأول مدينة'
  'إسكانية متكاملة في المملكة تلبي احتياجات المواطنين في الحصول على السكن.'
  
  )

question1 = "متى تم تأسيس لجنة الإسكان والتمليك"
question2 = "ما هي أول مدينة اسكانية تم تأسيسيها في البحرين"

In [30]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from src.finetuningQA.preprocess import ArabertPreprocessor

model = AutoModelForQuestionAnswering.from_pretrained("arabertv2-QA-model/")
tokenizer = AutoTokenizer.from_pretrained("arabertv2-QA-model/")

arabert_prep = ArabertPreprocessor(model_name=model_name)

context_ = arabert_prep.preprocess(context_1)
question_ = arabert_prep.preprocess(question1)

qa_model = pipeline("question-answering", model=model, tokenizer=tokenizer)
print("prediction {}".format(qa_model(question = question_, context = context_)))

print("The answer after unprocessing is : {}".format(arabert_prep.unpreprocess(qa_model(question = question_, context = context_)["answer"])))

[2022-08-27 22:32:45,736 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
prediction {'score': 0.9743682742118835, 'start': 269, 'end': 277, 'answer': 'عام 1962'}
The answer after unprocessing is : عام 1962
